In [3]:
import pandas as pd
import numpy as np

# Load dataset from the shared folder
file_path = 'FacoDMEK.xlsx'
df = pd.read_excel(file_path, sheet_name='Data', engine='openpyxl')

# Compute mean keratometry using Bio-Ks and Bio-Kf
# Some formulas use averaged K; this is one option. You may adjust if needed.
df['K_mean'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

# Function implementing SRK/T expected refraction using IOL power implanted

def srkt_expected_refraction(L, K_mean, A_const, IOL):
    """Compute expected postoperative spherical equivalent given axial length L (mm),
    mean keratometry K_mean (D), A-constant (dimensionless) and implanted IOL power (D)."""
    # Step 1: corneal radius (mm)
    r = 337.5 / K_mean
    # Step 2: corrected axial length (LCOR)
    if L <= 24.4:
        LCOR = L
    else:
        LCOR = 3.446 + 1.716 * L - 0.0237 * (L ** 2)
    # Step 3: corneal width
    Cw = -5.41 + 0.58412 * LCOR + 0.098 * K_mean
    # Step 4: corneal height (H)
    radicand = r**2 - (Cw**2) / 4
    if radicand < 0:
        radicand = 0
    H = r - np.sqrt(radicand)
    # Step 5: convert A constant to ACD constant
    ACD_const = 0.62467 * A_const - 68.747
    # Step 6: offset from iris plane to IOL optical plane
    offset = ACD_const - 3.336
    # Step 7: predicted postoperative anterior chamber depth
    ACD = H + offset
    # Step 8: constants and retinal thickness correction
    V = 12  # vertex distance in mm
    na = 1.336  # refractive index of aqueous/vitreous
    nem1 = 0.333  # n_e - 1 constant from paper
    RETHICK = 0.65696 - 0.02029 * L
    LOPT = L + RETHICK
    # Step 11: expected refraction (for given implanted IOL)
    numerator = 1000 * na * (na * r - nem1 * LOPT) - IOL * (LOPT - ACD) * (na * r - nem1 * ACD)
    denominator = na * (V * (na * r - nem1 * LOPT) + LOPT * r) - 0.001 * IOL * (LOPT - ACD) * (V * (na * r - nem1 * ACD) + ACD * r)
    return numerator / denominator

# Apply the SRK/T function row by row to compute expected refraction

df['SRKT_Pred'] = df.apply(lambda row: srkt_expected_refraction(row['Bio-AL'], row['K_mean'], row['A-Constant'], row['IOL Power']), axis=1)

# Calculate the error between predicted and actual postoperative spherical equivalent
df['Error'] = df['SRKT_Pred'] - df['PostOP Spherical Equivalent']

# Display the first few rows of the results
df[['ID', 'PostOP Spherical Equivalent', 'SRKT_Pred', 'Error']].head()

,ID,PostOP Spherical Equivalent,SRKT_Pred,Error
0,1,-3.875,0.740319,4.615319
1,2,-3.125,0.290750,3.415750
2,3,-3.000,-0.598300,2.401700
3,4,-2.250,-1.498023,0.751977
4,5,-2.250,-0.768112,1.481888


In [ ]:
# Compute summary statistics for prediction error
abs_errors = np.abs(df['Error'])
mae = abs_errors.mean()
rmse = np.sqrt(np.mean((df['Error'])**2))
print('Mean Absolute Error (MAE):', mae)
print('Root Mean Squared Error (RMSE):', rmse)

# Plot histogram of errors
import matplotlib.pyplot as plt
plt.hist(df['Error'], bins=15)
plt.xlabel('Prediction Error (D)')
plt.ylabel('Frequency')
plt.title('Distribution of SRK/T Prediction Error (using implanted IOL)')
plt.show()